In [1]:
from os import makedirs, listdir
from random import seed, random
from shutil import copyfile

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.applications import EfficientNetB0
from keras.callbacks import ReduceLROnPlateau
from keras import layers
from matplotlib.image import imread
from tensorflow.python.data import Dataset

print(f'TF version: {tf.__version__}')

Fontconfig warning: ignoring UTF-8: not a valid region tag


ModuleNotFoundError: No module named 'pandas'

In [51]:
img_size = 224

In [52]:
test: Dataset = tf.keras.utils.image_dataset_from_directory(
	directory='datasets/20210714/test',
	batch_size=16,
	image_size=(img_size,img_size)
)
train: Dataset = tf.keras.utils.image_dataset_from_directory(
	directory='datasets/20210714/train',
	batch_size=16,
	image_size=(img_size,img_size),
	validation_split=0.25,
	subset='training',
	seed=1
)
val: Dataset = tf.keras.utils.image_dataset_from_directory(
	directory='datasets/20210714/train',
	batch_size=16,
	image_size=(img_size,img_size),
	validation_split=0.25,
	subset='validation',
	seed=1
)

Found 669 files belonging to 63 classes.
Found 1549 files belonging to 63 classes.
Using 1162 files for training.
Found 1549 files belonging to 63 classes.
Using 387 files for validation.


In [53]:
base_model = EfficientNetB0(
	weights='imagenet',
	input_shape=(img_size,img_size,3),
	include_top=False,
)
base_model.trainable = False

In [54]:
def get_pretrained():
    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(img_size,img_size, 3))

    x = base_model(inputs)

    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    #Final Layer (Output)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[inputs], outputs=output)

    return model

In [55]:
model_pretrained = get_pretrained()
model_pretrained.compile(loss='categorical_crossentropy'
              , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005), metrics='categorical_accuracy')

model_pretrained.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d_6   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 128)               163968    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                           

In [56]:
batch_size = 32
n_epochs = 25

learning_rate_reduction = ReduceLROnPlateau(monitor='val_categorical_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

# with tf.device('/gpu:0'):
history_tl = model_pretrained.fit(
    train,
    epochs = n_epochs,
    callbacks = [learning_rate_reduction],
    steps_per_epoch = train.cardinality().numpy()/batch_size,
    validation_data = val
)

Epoch 1/25


2022-10-13 11:40:02.089097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/2 [=======================================] - ETA: 0s - loss: 0.0000e+00 - categorical_accuracy: 1.0000

2022-10-13 11:40:11.544063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 22s 11s/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000 - lr: 5.0000e-05
Epoch 2/25
2/2 [==============================] - 9s 6s/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000 - lr: 5.0000e-05
Epoch 3/25
3/2 [=======================================] - ETA: 0s - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.4999999621068127e-05.
2/2 [==============================] - 8s 6s/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000 - lr: 5.0000e-05
Epoch 4/25
2/2 [==============================] - 8s 6s/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000 - lr: 1.5000e-05
Epoch 5/25
3/2 [=======================================] - ETA: 0s - loss: 0.0000e

KeyboardInterrupt: 

In [ ]:
losses = pd.DataFrame(model_pretrained.history.history)
losses[['loss','val_loss']].plot()

In [ ]:
losses[['categorical_accuracy','val_categorical_accuracy']].plot()

In [ ]:
print("Loss of the model is - " , model_pretrained.evaluate(test)[0])
print("Accuracy of the model is - " , model_pretrained.evaluate(test)[1]*100 , "%")

## Fine-tuning

In [ ]:
base_model.trainable = True

# Retrain the last 10 layers (all lower layers will be kept frozen)
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [ ]:
model_pretrained.compile(loss='categorical_crossentropy'
              , optimizer = tf.keras.optimizers.Adam(learning_rate=0.000002), metrics='categorical_accuracy')

model_pretrained.summary()

In [ ]:
history_tl = model_pretrained.fit(
                    train,
                    epochs = n_epochs,
                    callbacks = [learning_rate_reduction],
					steps_per_epoch = train.cardinality().numpy()/batch_size,
					validation_data = val
                   )

## Results

Test loss & accuracy

In [ ]:
print("Loss of the model is - " , model_pretrained.evaluate(test)[0])
print("Accuracy of the model is - " , model_pretrained.evaluate(test)[1]*100 , "%")